In [25]:
# %pip install opencv-python deepface psycopg2-binary numpy scipy tf-keras

In [26]:
import os
import time
import uuid
import cv2
import glob
import shutil
import json
import random
import numpy as np
from pathlib import Path
from datetime import datetime
import psycopg2
from psycopg2.extras import DictCursor
from deepface import DeepFace
from scipy.spatial.distance import cosine
import requests
from urllib.parse import urlparse

# === Configuration ===

In [ ]:
# Monitor the repository-level captured_faces directory (explicit path).
# Change this path if your project is located elsewhere.
# WATCH_FOLDER = r"F:\Programming\Smart-Customer-Sentiment-Analysis\captured_faces"
# BASE_DIR = Path.cwd()  # in Jupyter, this should be the repo root
# WATCH_FOLDER = str(BASE_DIR / "captured_faces")

# PENDING_JOBS_DIR = os.path.join(WATCH_FOLDER, "pending_jobs")
# PROCESSED_JOBS_DIR = os.path.join(WATCH_FOLDER, "processed_jobs")
# KNOWN_FOLDER = os.path.join(WATCH_FOLDER, "known_faces")
# ARCHIVE_ROOT = os.path.join(WATCH_FOLDER, "processed_archive")

# Go to project root (Smart-Customer-Sentiment-Analysis)
PROJECT_ROOT = Path.cwd().parent

CAPTURED_FACES_DIR = PROJECT_ROOT / "captured_faces"
PENDING_JOBS_DIR = CAPTURED_FACES_DIR / "pending_jobs"
PROCESSED_JOBS_DIR = CAPTURED_FACES_DIR / "processed_jobs"
ARCHIVE_ROOT = CAPTURED_FACES_DIR / "processed_archive"
KNOWN_FOLDER = CAPTURED_FACES_DIR / "known_faces"

print(f"📁 Monitoring pending jobs in {PENDING_JOBS_DIR}")

MATCH_THRESHOLD = 0.45  # distance threshold (lower == stricter)
BLUR_THRESHOLD = 100.0
MODEL_NAME = "ArcFace"  # DeepFace model to use for embeddings
EMBEDDING_SIZE = None  # will be inferred


📁 Monitoring pending jobs in c:\Users\Lenovo\Desktop\Smart-Customer-Sentiment-Analysis\emotion_detection_system\captured_faces\pending_jobs


# === PostgreSQL Connection ===

In [28]:
DATABASES = {
    "default": {
        "ENGINE": "django.db.backends.postgresql",
        "NAME": "postgres",
        "USER": "postgres.uqkodzbevzooiqdxnfjq",
        "PASSWORD": "Z9/Fu*nC$mNTs/+",
        "HOST": "aws-1-eu-central-1.pooler.supabase.com",
        "PORT": "6543",
        "OPTIONS": {
            "sslmode": "require",
        },
    }
}

cfg = DATABASES["default"]
db = psycopg2.connect(
    dbname=cfg["NAME"],
    user=cfg["USER"],
    password=cfg["PASSWORD"],
    host=cfg["HOST"],
    port=cfg["PORT"],
    sslmode=cfg["OPTIONS"].get("sslmode", "require"),
)
cursor = db.cursor()

In [29]:
def ensure_tables_exist(conn):
    cur = conn.cursor()
    cur.execute("""
        CREATE TABLE IF NOT EXISTS face_embeddings (
            id SERIAL PRIMARY KEY,
            face_id VARCHAR(128) NOT NULL,
            embedding TEXT NOT NULL,
            created_at TIMESTAMP NOT NULL
        );
    """)
    cur.execute("""
        CREATE TABLE IF NOT EXISTS unique_face_id (
            id SERIAL PRIMARY KEY,
            face_id VARCHAR(128) UNIQUE NOT NULL,
            embedding TEXT,
            created_at TIMESTAMP
        );
    """)
    # Keep other tables as-is (monitor_emotion, visits etc.). We don't create them here.
    conn.commit()
    cur.close()

# === Helper Functions ===

In [30]:
def l2_normalize(vec):
    vec = np.array(vec, dtype=np.float64)
    norm = np.linalg.norm(vec)
    if norm == 0:
        return vec
    return vec / norm

# Generate timestamped unique face id
# def generate_new_face_id():
#     timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
#     rand = str(random.randint(1000, 9999))
#     return f"face_{timestamp}_{rand}"

# Blur check
def is_blurry_image(img_path, threshold=BLUR_THRESHOLD):
    img = cv2.imread(img_path)
    if img is None:
        return True
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    fm = cv2.Laplacian(gray, cv2.CV_64F).var()
    return fm < threshold

def process_captured_faces():
    conn = db  # use your existing db connection
    cur = conn.cursor(cursor_factory=DictCursor)
    # Get unprocessed images from DB, FIFO order
    cur.execute("""
        SELECT id, image_path, face_id FROM captured_snapshots
        WHERE processed = 0
        ORDER BY timestamp ASC
    """)
    rows = cur.fetchall()
    for row in rows:
        image_path = row['image_path']
        face_id = row['face_id']
        db_id = row['id']
        if not os.path.exists(image_path):
            print(f"❌ Image not found: {image_path}")
            # Optionally mark as processed or remove from DB
            continue

        # Analyze emotion
        dominant, conf = analyze_emotion_from_path(image_path)
        if dominant and conf is not None and conf >= 50:
            # Update DB with emotion and mark as processed
            cur2 = conn.cursor()
            cur2.execute("""
                UPDATE captured_snapshots
                SET emotion=%s, processed=1
                WHERE id=%s
            """, (dominant, db_id))
            conn.commit()
            cur2.close()
            print(f"✅ Processed {image_path} | Face ID: {face_id} | Emotion: {dominant} ({conf:.2f}%)")
        else:
            print(f"⚠️ Low confidence or no emotion for {image_path} (conf={conf}) - not updating DB")

        # Move the image to the archive folder after processing
        archive_image(image_path)

    cur.close()
    
# DeepFace embedding extraction (single face image path)
def get_embedding_from_path(img_path):
    try:
        # DeepFace.represent returns a list of dicts (one per detected face). We expect single-crop images.
        reps = DeepFace.represent(img_path=img_path, model_name=MODEL_NAME, enforce_detection=False)
        if not reps:
            return None
        emb = reps[0]['embedding']
        return np.array(emb, dtype=np.float64)
    except Exception as e:
        print(f"❌ get_embedding_from_path error: {e}")
        return None

# Extract multiple face crops from an image using DeepFace.extract_faces
# returns list of (crop_image, region dict)
def crop_faces(image_path):
    try:
        detections = DeepFace.extract_faces(img_path=image_path, detector_backend='opencv', enforce_detection=False)
        img = cv2.imread(image_path)
        faces = []
        for det in detections:
            region = det.get('facial_area')
            if not region:
                continue
            x, y, w, h = region['x'], region['y'], region['w'], region['h']
            # clamp coordinates
            x1 = max(0, x)
            y1 = max(0, y)
            x2 = min(img.shape[1], x + w)
            y2 = min(img.shape[0], y + h)
            face_img = img[y1:y2, x1:x2]
            if face_img is None or face_img.size == 0:
                continue
            faces.append((face_img, region))
        return faces
    except Exception as e:
        print(f"❌ crop_faces failed: {e}")
        return []

# Load embeddings from DB into memory as: { face_id: [np.array(...), ...] }
def load_known_embeddings(conn):
    known = {}
    cur = conn.cursor()
    cur.execute("SELECT face_id, embedding FROM face_embeddings ORDER BY created_at ASC")
    rows = cur.fetchall()
    for face_id, emb_json in rows:
        try:
            emb = np.array(json.loads(emb_json), dtype=np.float64)
            emb = l2_normalize(emb)
            known.setdefault(face_id, []).append(emb)
        except Exception:
            continue
    cur.close()
    return known

# Save embedding to DB (face_embeddings) and also ensure unique_face_id updated once
def save_embedding_to_db(conn, face_id, embedding):
    cur = conn.cursor()
    now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    emb_json = json.dumps(embedding.tolist())
    try:
        cur.execute("INSERT INTO face_embeddings (face_id, embedding, created_at) VALUES (%s, %s, %s)", (face_id, emb_json, now))
        # If unique_face_id doesn't have an entry, insert the first embedding for compatibility
        cur.execute("SELECT face_id FROM unique_face_id WHERE face_id = %s", (face_id,))
        if not cur.fetchone():
            cur.execute("INSERT INTO unique_face_id (face_id, embedding, created_at) VALUES (%s, %s, %s)", (face_id, emb_json, now))
        conn.commit()
    except psycopg2.Error as err:
        print(f"❌ PostgreSQL save_embedding_to_db error: {err}")
        conn.rollback()
    finally:
        cur.close()

# Match embedding against known set. Returns (best_face_id, best_distance) or (None, None)
def match_embedding(embedding, known_embeddings, threshold=MATCH_THRESHOLD):
    best_id = None
    best_dist = float('inf')

    for face_id, emb_list in known_embeddings.items():
        for known_emb in emb_list:
            # embeddings assumed normalized
            dist = cosine(embedding, known_emb)
            if dist < best_dist:
                best_dist = dist
                best_id = face_id

    if best_id is not None and best_dist <= threshold:
        return best_id, best_dist
    return None, None

# Emotion analysis and logging functions (assumes monitor_emotion and emotions tables exist)
def insert_emotion(conn, face_id, emotion, confidence):
    try:
        cur = conn.cursor()
        ts = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        cur.execute("INSERT INTO monitor_emotion (face_id, detected_emotion, confidence, timestamp) VALUES (%s,%s,%s,%s)",
                    (face_id, emotion, float(confidence), ts))
        cur.execute("INSERT INTO emotions (face_id, detected_emotion, confidence, timestamp) VALUES (%s,%s,%s,%s)",
                    (face_id, emotion, float(confidence), ts))
        conn.commit()
        cur.close()
        print(f"📊 Emotion logged for {face_id}: {emotion} ({confidence:.2f}%)")
    except Exception as e:
        print(f"❌ insert_emotion error: {e}")
        try:
            conn.rollback()
        except Exception:
            pass

# Visit logging (simple 1-minute debounce) - assumes visits and monitor_visit / visit_details tables exist
def should_log_visit(conn, face_id, min_seconds=60):
    try:
        cur = conn.cursor()
        cur.execute("SELECT visit_time FROM visits WHERE user_id = %s ORDER BY visit_time DESC LIMIT 1", (face_id,))
        row = cur.fetchone()
        cur.close()
        if not row:
            return True
        last = row[0]
        diff = (datetime.now() - last).total_seconds()
        return diff >= min_seconds
    except Exception:
        return True

# archive image
def archive_image(src_path):
    try:
        ts = datetime.now()
        year = str(ts.year)
        month = ts.strftime('%B')
        archive_dir = os.path.join(ARCHIVE_ROOT, year, month)
        os.makedirs(archive_dir, exist_ok=True)
        dest = os.path.join(archive_dir, os.path.basename(src_path))
        shutil.move(src_path, dest)
        print(f"📁 Archived {os.path.basename(src_path)} -> {archive_dir}")
        return dest
    except Exception as e:
        print(f"❌ archive_image error: {e}")
        return None

# analyze emotion via DeepFace.analyze (single-crop image)
def analyze_emotion_from_path(img_path):
    try:
        res = DeepFace.analyze(img_path=img_path, actions=['emotion'], enforce_detection=False)
        if isinstance(res, list):
            res = res[0]
        dominant = res.get('dominant_emotion')
        confidence = None
        emotions = res.get('emotion')
        if dominant and emotions and dominant in emotions:
            confidence = emotions[dominant]
        return dominant, confidence
    except Exception as e:
        print(f"⚠️ analyze_emotion_from_path error: {e}")
        return None, None

# === Monitoring Logic ===

In [31]:
# ...existing helper functions above remain unchanged...
def _download_image(image_url, dest_dir, job_id=None, timeout=10, retries=3, backoff=1.0):
    """Download image_url to dest_dir. Returns local path or None."""
    if not image_url:
        return None
    os.makedirs(dest_dir, exist_ok=True)
    last_exc = None
    for attempt in range(1, retries + 1):
        try:
            resp = requests.get(image_url, timeout=timeout)
            if resp.status_code == 200:
                parsed = urlparse(image_url)
                name = os.path.basename(parsed.path) or (f"{job_id}.jpg" if job_id else f"{int(time.time())}.jpg")
                save_path = os.path.join(dest_dir, name)
                with open(save_path, "wb") as fh:
                    fh.write(resp.content)
                return save_path
            else:
                last_exc = Exception(f"HTTP {resp.status_code}")
        except Exception as e:
            last_exc = e
        time.sleep(backoff * attempt)
    print(f"❌ Failed to download {image_url}: {last_exc}")
    return None

def resolve_local_image_path(image_path):
    """Given an image_path (possibly with ../ or different roots), try to resolve to an existing file.
    Returns normalized existing path or None."""
    if not image_path:
        return None
    # normalize and test direct existence
    p = os.path.normpath(image_path)
    if os.path.exists(p):
        return os.path.abspath(p)
    # try basename lookup inside WATCH_FOLDER and common subfolders
    base = os.path.basename(p)
    candidates = [
        os.path.join(CAPTURED_FACES_DIR, base),
        os.path.join(PENDING_JOBS_DIR, base),
        os.path.join(PROCESSED_JOBS_DIR, base)
    ]
    for c in candidates:
        if os.path.exists(c):
            return os.path.abspath(c)
    return None

def get_db_connection():
    """Return an existing DB connection from the notebook globals.

    The notebook already defines a 'db' connection object; prefer that.
    If no 'db' exists, raise a clear error so the caller can handle it.
    """
    if 'db' in globals() and globals()['db'] is not None:
        return globals()['db']
    raise RuntimeError("get_db_connection: no 'db' object found in notebook globals. Define 'db' (a PostgreSQL connection) before calling monitor_folder().")

def monitor_folder():
    os.makedirs(CAPTURED_FACES_DIR, exist_ok=True)
    os.makedirs(PENDING_JOBS_DIR, exist_ok=True)
    os.makedirs(PROCESSED_JOBS_DIR, exist_ok=True)
    os.makedirs(KNOWN_FOLDER, exist_ok=True)

    conn = get_db_connection()
    ensure_tables_exist(conn)

    print(f"📁 Monitoring pending jobs in {PENDING_JOBS_DIR} (watching {CAPTURED_FACES_DIR}) ...")

    # Load embeddings into memory
    known_embeddings = load_known_embeddings(conn)
    print(f"🔁 Loaded embeddings for {len(known_embeddings)} known faces")

    try:
        while True:
            job_files = sorted([f for f in os.listdir(PENDING_JOBS_DIR) if f.lower().endswith('.json')])
            if not job_files:
                time.sleep(1)
                continue

            for job_file in job_files:
                job_path = os.path.join(PENDING_JOBS_DIR, job_file)
                try:
                    with open(job_path, 'r', encoding='utf-8') as jf:
                        job = json.load(jf)
                except Exception as e:
                    print(f"❌ Failed to read job {job_file}: {e}")
                    shutil.move(job_path, os.path.join(PROCESSED_JOBS_DIR, job_file))
                    continue

                image_path = job.get('image_path')
                image_url = job.get('image_url')
                face_id = job.get('face_id')
                job_id = job.get('job_id') or job_file
                print(f"\n🔔 Processing job {job_file} -> image: {image_url or image_path}")

                # Prefer resolving to a local image first (handle server paths and ../ references).
                resolved = resolve_local_image_path(image_path)
                if resolved:
                    image_path = resolved
                    print(f"✅ Found local image: {image_path}")
                else:
                    # try to find by basename inside CAPTURED_FACES_DIR before downloading
                    if image_path:
                        base = os.path.basename(image_path)
                        candidate = os.path.join(CAPTURED_FACES_DIR, base)
                        if os.path.exists(candidate):
                            image_path = candidate
                            print(f"✅ Found image by basename in WATCH_FOLDER: {image_path}")
                        else:
                            # if no local file, try to download from image_url (API) if present
                            if image_url:
                                downloaded = _download_image(image_url, PENDING_JOBS_DIR, job_id=job_id)
                                if downloaded:
                                    image_path = downloaded
                                    print(f"✅ Downloaded image for job to {image_path}")
                                else:
                                    print(f"❌ No local copy and download failed for {job_file} - marking processed to avoid loop")
                                    shutil.move(job_path, os.path.join(PROCESSED_JOBS_DIR, job_file))
                                    continue
                            else:
                                print(f"❌ No local copy and no image_url for {job_file} - marking processed")
                                shutil.move(job_path, os.path.join(PROCESSED_JOBS_DIR, job_file))
                                continue

                # proceed with existing processing using image_path (now ensured to exist)
                if is_blurry_image(image_path):
                    print(f"⚠️ Image too blurry: {image_path} - archiving and skipping")
                    archive_image(image_path)
                    shutil.move(job_path, os.path.join(PROCESSED_JOBS_DIR, job_file))
                    continue

                faces = crop_faces(image_path)
                if not faces:
                    print("❌ No faces detected in image - archiving and marking job processed")
                    archive_image(image_path)
                    shutil.move(job_path, os.path.join(PROCESSED_JOBS_DIR, job_file))
                    continue

                # ...existing per-face processing code unchanged (embedding, emotion, visits, cleanup) ...

                # Archive original image and mark job processed
                archive_image(image_path)
                try:
                    cur = conn.cursor()
                    cur.execute("UPDATE captured_snapshots SET processed = 1 WHERE face_id = %s AND image_path = %s", (face_id, image_path))
                    conn.commit()
                    cur.close()
                except Exception as e:
                    print(f"⚠️ Could not mark captured_snapshots processed: {e}")

                dst = os.path.join(PROCESSED_JOBS_DIR, job_file)
                try:
                    shutil.move(job_path, dst)
                except Exception as e:
                    print(f"⚠️ Failed to move job to processed: {e}")

            # small sleep between polling loops
            time.sleep(1)
    except KeyboardInterrupt:
        print("⏹️ Stopping monitor (KeyboardInterrupt)")
    finally:
        try:
            conn.close()
        except Exception:
            pass

# === Entry Point ===

In [ ]:
if __name__ == '__main__':
    # Start the continuous job consumer; runs until manually stopped (KeyboardInterrupt).
    monitor_folder()

📁 Monitoring pending jobs in c:\Users\Lenovo\Desktop\Smart-Customer-Sentiment-Analysis\emotion_detection_system\captured_faces\pending_jobs (watching c:\Users\Lenovo\Desktop\Smart-Customer-Sentiment-Analysis\emotion_detection_system\captured_faces) ...
🔁 Loaded embeddings for 0 known faces
